## KNN Algorithm 

### 使用airbnb做房價預測

<img src="1.png" style="width:800px;height:480px;float:left">

## 資料抓取

In [3]:
import pandas as pd

features = ['accommodates','bedrooms','bathrooms','beds','price','minimum_nights','maximum_nights','number_of_reviews']
#篩選出有用的特徵

dc_listings = pd.read_csv('listings2.csv',low_memory=False)

dc_listings = dc_listings[features]

print(dc_listings.shape)

dc_listings.head()

(8409, 8)


accommodates  bedrooms  bathrooms  beds      price  minimum_nights  \
0             2       1.0        1.0   1.0  $1,498.00               7   
1             2       1.0        1.0   1.0    $898.00              30   
2             3       1.0        1.0   1.0  $1,496.00               3   
3             2       1.0        1.5   1.0  $1,258.00               3   
4             1       1.0        4.0   4.0    $389.00               1   

   maximum_nights  number_of_reviews  
0             365                  2  
1             365                  7  
2              28                 49  
3             365                 52  
4              90                 77

擷取的特徵

* accommodates: 可以容納的房客
* bedrooms: 房間的數量
* bathrooms: 廁所的數量
* beds: 床的數量
* price: 一晚的費用
* minimum_nights: 客人最少租了幾天
* maximum_nights: 客人最多租了幾天
* number_of_reviews: 評論數

## 如果我今天他媽的有一張床，沒有女生要住我們家，我能租多少錢?
--> 484要看看別人都賣多少


<img src="2.png" style="width:600px;height:230px;float:left">

K代表我们的候选对象个数，也就是找和我房间数量最相近的其他房子的价格

## K近邻原理

<img src="3.png" style="width:600px;height:330px;float:left">

假设我们的数据源中只有5条信息，现在我想针对我的房子（只有一个房间）来定一个价格。

<img src="4.png" style="width:600px;height:330px;float:left">

在这里假设我们选择的K=3，也就是选3个跟我最相近的房源。

<img src="5.png" style="width:600px;height:330px;float:left">

再综合考虑这三个我就得到了我的房子大概能值多钱啦！

## 距离的定义
如何才能知道哪些数据样本跟我最相近呢？

<img src="6.png" style="width:400px;height:80px;float:left">

其中Q1到Qn是一条数据的所有特征信息，P1到Pn是另一条数据的所有特征信息

假设我们的房子有3个房间

In [5]:
import numpy as np

our_acc_value = 3

dc_listings['distance'] = np.abs(dc_listings.accommodates - our_acc_value)
dc_listings.distance.value_counts().sort_index()

0      777
1     4737
2     1276
3      653
4      146
5      319
6       59
7      123
8       24
9       88
10      16
11      28
12      15
13     148
Name: distance, dtype: int64

这里我们只有了绝对值来计算，和我们距离为0的（同样数量的房间）有461个

sample操作可以得到洗牌后的数据

In [6]:
dc_listings = dc_listings.sample(frac=1,random_state=0)
dc_listings = dc_listings.sort_values('distance')
dc_listings.price.head()

2330      $509.00
1592    $1,736.00
4581    $2,514.00
8142    $3,685.00
7793    $1,107.00
Name: price, dtype: object

现在的问题是，这里面的数据是字符串呀，需要转换一下！

In [7]:
dc_listings['price'] = dc_listings.price.str.replace("\$|,",'').astype(float)

mean_price = dc_listings.price.iloc[:5].mean()
mean_price

1910.2

得到了平均价格，也就是我们的房子大致的价格了

## 模型的评估

<img src="7.png" style="width:600px;height:250px;float:left">

首先制定好训练集和测试集

In [8]:
dc_listings.drop('distance',axis=1)

train_df = dc_listings.copy().iloc[:2792]
test_df = dc_listings.copy().iloc[2792:]

基于单变量预测价格

In [9]:
def predict_price(new_listing_value,feature_column):
    temp_df = train_df
    temp_df['distance'] = np.abs(dc_listings[feature_column] - new_listing_value)
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return(predicted_price)

In [10]:
test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column='accommodates')


这样我们就得到了测试集中，所以房子的价格了

root mean squared error (RMSE)均方根误差

<img src="8.png" style="width:700px;height:100px;float:left">

In [11]:
test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
mse = test_df['squared_error'].mean()
rmse = mse ** (1/2)
rmse

10128.775096721143

现在我们得到了对于一个变量的模型评估得分

## 不同的变量效果会不会不同呢？

In [12]:
for feature in ['accommodates','bedrooms','bathrooms','number_of_reviews']:
    #test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column=feature)
    test_df['predicted_price'] = test_df[feature].apply(predict_price,feature_column=feature)
    test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
    mse = test_df['squared_error'].mean()
    rmse = mse ** (1/2)
    print("RMSE for the {} column: {}".format(feature,rmse))

RMSE for the accommodates column: 10128.775096721143
RMSE for the bedrooms column: 10152.41986962611
RMSE for the bathrooms column: 10200.366737765708
RMSE for the number_of_reviews column: 10221.967555276657


看起来结果差异还是蛮大的，接下来我们要做的就是综合利用所有的信息来一起进行测试

In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
features = ['accommodates','bedrooms','bathrooms','beds','price','minimum_nights','maximum_nights','number_of_reviews']

dc_listings = pd.read_csv('listings.csv')

dc_listings = dc_listings[features]

dc_listings['price'] = dc_listings.price.str.replace("\$|,",'').astype(float)

dc_listings = dc_listings.dropna()

dc_listings[features] = StandardScaler().fit_transform(dc_listings[features])

normalized_listings = dc_listings

print(dc_listings.shape)

normalized_listings.head()

(3671, 8)


accommodates  bedrooms  bathrooms      beds     price  minimum_nights  \
0      0.401420 -0.249501  -0.439211  0.297386  0.081119       -0.341421   
1      1.399466  2.129508   2.969551  1.141704  1.462622       -0.065047   
2     -1.095648 -0.249501   1.265170 -0.546933 -0.718699       -0.065047   
3     -0.596625 -0.249501  -0.439211 -0.546933 -0.391501       -0.341421   
4      0.401420 -0.249501  -0.439211 -0.546933 -0.718699        1.316824   

   maximum_nights  number_of_reviews  
0       -0.016575          -0.516779  
1       -0.016606           1.706767  
2       -0.016575          -0.482571  
3       -0.016575          -0.516779  
4       -0.016575          -0.516779

In [14]:
norm_train_df = normalized_listings.copy().iloc[0:2792]
norm_test_df = normalized_listings.copy().iloc[2792:]

多变量距离的计算

<img src="9.png" style="width:700px;height:400px;float:left">

scipy中已经有现成的距离的计算工具了

In [15]:
from scipy.spatial import distance

first_listing = normalized_listings.iloc[0][['accommodates', 'bathrooms']]
fifth_listing = normalized_listings.iloc[20][['accommodates', 'bathrooms']]
first_fifth_distance = distance.euclidean(first_listing, fifth_listing)
first_fifth_distance

3.723019604017032

## 多变量KNN模型

In [16]:
def predict_price_multivariate(new_listing_value,feature_columns):
    temp_df = norm_train_df
    temp_df['distance'] = distance.cdist(temp_df[feature_columns],[new_listing_value[feature_columns]])
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return(predicted_price)

cols = ['accommodates', 'bathrooms']
norm_test_df['predicted_price'] = norm_test_df[cols].apply(predict_price_multivariate,feature_columns=cols,axis=1)    
norm_test_df['squared_error'] = (norm_test_df['predicted_price'] - norm_test_df['price'])**(2)
mse = norm_test_df['squared_error'].mean()
rmse = mse ** (1/2)
print(rmse)

0.7894063922577537


In [19]:
#norm_test_df['price'] = norm_test_df.price.str.replace("\$|,",'').astype(float)
#https://blog.csdn.net/FrankieHello/article/details/79659111
mean_price = norm_test_df.price.iloc[:5].mean()
scaler.inverse_transform(mean_price)

NameError: name 'inverse_transform' is not defined

## 使用Sklearn来完成KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
cols = ['accommodates','bedrooms']
knn = KNeighborsRegressor()
knn.fit(norm_train_df[cols], norm_train_df['price'])
two_features_predictions = knn.predict(norm_test_df[cols])

In [ ]:
from sklearn.metrics import mean_squared_error

two_features_mse = mean_squared_error(norm_test_df['price'], two_features_predictions)
two_features_rmse = two_features_mse ** (1/2)
print(two_features_rmse)

加入更多的特征

In [ ]:
knn = KNeighborsRegressor(5)

cols = ['accommodates','bedrooms','bathrooms','beds','minimum_nights','maximum_nights','number_of_reviews']

knn.fit(norm_train_df[cols], norm_train_df['price'])
four_features_predictions = knn.predict(norm_test_df[cols])
four_features_mse = mean_squared_error(norm_test_df['price'], four_features_predictions)
four_features_rmse = four_features_mse ** (1/2)
four_features_rmse


#參考資料
https://www.twblogs.net/a/5baab1662b7177781a0e63a0
https://blog.csdn.net/weixin_41789707/article/details/80930274?utm_source=copy